In [ ]:
# 載入使用的套件
import pandas as pd
import requests
import json
import re
infos = []
# 撰寫簡單的函數，透過輸入文章ID，就輸出文章的資料
def Crawl(ID):
  info = {}
  link = 'https://www.dcard.tw/_api/posts/' + str(ID)
  requ = requests.get(link)
  rejs = json.loads(requ.text)
  for d in rejs:
    info['ID'] = d['id']
    info['title'] = d['title']
    info['excerpt'] = d['excerpt']
    info['forumName'] = d['forumName']
  infos.append(info)



url = 'https://www.dcard.tw/_api/posts?limit=100'
resq = requests.get(url)
rejs = json.loads(resq.text)
for d in rejs:
  info = {}
  info['ID'] = d['id']
  info['title'] = d['title']
  info['excerpt'] = d['excerpt']
  info['forumName'] = d['forumName']
  infos.append(info)

df = pd.DataFrame(infos)
df.columns = ['ID','title','excerpt','forumName']
print(df)
df.to_csv('dcard.csv', encoding='utf_8_sig')  

In [ ]:
# 載入使用的套件
import pandas as pd
import requests
import json
import re
infos = []
# 撰寫簡單的函數，透過輸入文章ID，就輸出文章的資料
def Crawl(ID):
  info = {}
  link = 'https://www.dcard.tw/_api/posts/' + str(ID)
  requ = requests.get(link)
  rejs = json.loads(requ.text)
  for d in rejs:
    info['ID'] = d['id']
    info['title'] = d['title']
    info['excerpt'] = d['excerpt']
    info['forumName'] = d['forumName']
  infos.append(info)



url = 'https://www.dcard.tw/_api/posts?limit=100'
resq = requests.get(url)
rejs = json.loads(resq.text)
for d in rejs:
  info = {}
  info['ID'] = d['id']
  info['title'] = d['title']
  info['excerpt'] = d['excerpt']
  info['forumName'] = d['forumName']
  infos.append(info)

df = pd.DataFrame(infos)
df.columns = ['ID','title','excerpt','forumName']
print(df)

In [38]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [23]:
#hw2
import multiprocessing
from multiprocessing import Process, Queue

import time
from lxml import etree
import requests
from bs4 import BeautifulSoup

#url = 'https://movies.yahoo.com.tw/moviegenre_result.html?genre_id=1'

infos = []


def check_req_url(url): #測試請求網址是否請求成功
    r = requests.get(url) #請求網址
    #print(resp.status_code) #錯誤時404,成功時200
    if r.status_code != 200:  #如果請求失敗
        print('Invalid url:', r.url) #印出請求失敗的網址
        return "fail" #回傳失敗提示訊息
    else:
        return r.text #回傳請求成功的html文字

def get_news_info(webpage): #抓取電影資訊
  soup = BeautifulSoup(webpage) #網頁解析
  exist = soup.find_all('div', id = 'content_l')
  if exist:
    for d in soup.find_all('div', id="content_l"):
        info = {}
        info['中文名稱'] = d.find('div', class_='movie_intro_info_r').find('h1').text.strip()
        info['英文名稱'] = d.find('div', class_='movie_intro_info_r').find('h3').text.strip()
        info['上映日期'] = d.find('span').text.strip()
        info['劇情介紹'] = d.find('div', class_='gray_infobox storeinfo').find('div', class_='gray_infobox_inner').find('span',id='story').text.strip()
        # info['連接介紹'] = d.find('div', class_='release_movie_name').find('a', class_='gabtn').get('href')
        # net = d.find('div', class_='release_movie_name').find('a', class_='gabtn').get('href')
        print(info['中文名稱'])
        if d.find('div', class_='movie_intro_info_r').find('div',class_='level_name'):
          info['oneLabel'] = d.find('div', class_='movie_intro_info_r').find('div',class_='level_name').a.text.strip()
        else:
          info['oneLabel'] = 'NA'
        # print(info['oneLabel'])
        level_name = d.find('div', class_='movie_intro_info_r').find_all('div',class_='level_name')
        str =''
        for level in level_name:
          str += level.a.text.strip()+',' 
        info['Label'] = str
           
        infos.append(info) 
    return infos
  else:
    return 0

urlarray = []
if __name__ == '__main__':
  url = 'https://tw.news.yahoo.com/'
  r = requests.get(url)
  print(r)
  soup = BeautifulSoup(r.text) #網頁解析
  for d in soup.find('div', class_ = '_yb_14rjq _yb_vb2n7 '):
    print("in soup.find_all")
    urltemp = {}   
    urltemp['list_url'] = d.find('div',class_='ybar-mod-navigation _yb_1bzrv    _yb_1p468 ybar-shift-more-menu ').find_all('li',class_='_yb_1fp89 _yb_12rh5').find.a.get('href')
    print(urltemp['list_url'])
    urltemp['list_name'] = d.find_all('li',class_='_yb_1fp89 _yb_12rh5').find('div',class_='_yb_1fibg').text.strip()
    print("urltemp['list_name'] = ")
    print(urltemp['list_name'])
    urlarray.append(urltemp)

for i in urlarray:
  print(i)

  # for i in urlarray:
  #   # 'https://movies.yahoo.com.tw/movieinfo_main/%d'%(i)+''
  #   url = i
  #   r = requests.get(url, headers=headers)
  #   soup = BeautifulSoup(r.text) #網頁解析
  #   if soup.find_all('div', class_='movie_intro_info_r'):
  #     infos = get_week_new_movies(r.text)
  #   else:
  #     print("NON")
  #     continue
  # import pandas as pd

  # df = pd.DataFrame(infos)
  # df.to_csv('movies.csv', encoding='utf_8_sig')     
      
  # df    
    

<Response [200]>
in soup.find_all


AttributeError: ignored

In [ ]:
#hw3

import re
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import jieba
import jieba.analyse
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.model_selection import train_test_split    #分割資料集

def cutval(df):
    topcut = []
    for d in df['劇情介紹']:
        top = jieba.analyse.extract_tags(d,topK=2)
        topcut.append(top)
    typecut = []
    for d in df['oneLabel']:
        # ch =re.compile("[\u4e00-\u9fa5]")
        # seg_word =  "".join(ch.findall(d))
        # top = jieba.lcut(seg_word)
        typecut.append(d)
    # print(typecut)
    namecut = []
    for d in df['中文名稱']:
        ch =re.compile("[\u4e00-\u9fa5]")
        name =  "".join(ch.findall(d))
        # print("name = ")
        # print(name )
        namecut.append(name)
    # print(namecut)
    data = { 'type':typecut ,'name':namecut,'article':topcut}
    df1 = pd.DataFrame(data)

    df1['type'] = df1['type'].apply(lambda x :  str(x)[1:-2].replace("'","").replace(",",""))
    df1['article'] = df1['article'].apply(lambda x :  str(x)[1:-2].replace("'","").replace(",",""))
    df1['key'] = df1['name'].astype(str)+' '+df1['type'].astype(str)+' '+df1['article'].astype(str)
    return df1

def count(df):
    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(df['key'])
    tfidf = TfidfTransformer() 
    tf=tfidf.fit_transform(X)
    word = vectorizer.get_feature_names() #詞袋 
    print("tf = ")
    print(tf)
    return tf

def knn(X,Y):
    X_train, X_test, y_train, y_test = train_test_split(X, Y['type'].str[0:2], test_size=0.07)
    clf=KNeighborsClassifier(n_neighbors=51)
    clf.fit(X_train,y_train)

    y_pred = clf.predict(X_test)    # 預測模型 
    y_test=y_test.values
    print("精準度：",metrics.accuracy_score(y_test, y_pred))

df = pd.read_csv('/content/drive/MyDrive/NatureLanguageProgram/B0729064_NLP_HW3/movies.csv')
df = df.drop(labels=['Unnamed: 0'],axis='columns')
df1 = cutval(df) 
tf = count(df1)
knn(tf,df1)

In [18]:

import requests

from bs4 import BeautifulSoup

from urllib.parse import quote



###############################################################################

#                       股票機器人 爬蟲範例 【html爬蟲】                      #

###############################################################################



def getYahooNews(keyword):

    url = 'https://tw.finance.yahoo.com/news_search.html?ei=Big5&q=' + quote(keyword.encode('big5'))

    #請求網站

    list_req = requests.get(url)

    #將整個網站的程式碼爬下來

    soup = BeautifulSoup(list_req.content, "html.parser")
    getNew= soup.find('table',{'id':'newListTable'}) #抓到新聞的table
    getAllNews=getNew.find_all('table');

    

    for i in range(0,len(getAllNews)):

        onebody=getAllNews[i];

        NewTitle=onebody.find_all('a',{'class':'mbody'});

        NewsDt=onebody.find('span',{'class':'t1'});

        if (len(NewTitle) >= 1):

            print("文章標題:"+NewTitle[1].text+NewsDt.text)

        Context=onebody.find_all('span',{'class':'mbody'});

        if(len(Context)>=1):

            print("內文:"+Context[0].text)

        print('');



getYahooNews('中鋼')

文章標題:【台股盤後】航運半導體雙主流領軍 台股強彈399.9點（中央社 2021-08-23）
內文:（中央社記者潘智義台北2021年8月23日電）美股反彈帶動台積電(2330)、聯電(2303)等半導體族群走揚，攜手散裝及貨櫃族群強漲，盤中大漲逾400點；指數終場大漲399.9點，以16741.84


文章標題:電子、航運領軍，台股突破性大漲，挑戰半年線（財訊快報 2021-08-23）
內文:【財訊快報／研究員鄭心丹】美股四大指數上周五收紅，道瓊上漲225.96點，Fed卡普蘭意外「微調」鷹派觀點，科技股走高，微軟、思科、Salesforce領軍大漲之下，那斯達克漲幅1.19%。亞股周一全


文章標題:豐興開出本週盤價，鋼筋、廢鋼及型鋼產品維持平盤（財訊快報 2021-08-23）
內文:...公噸價格2.26萬元；廢鋼每公噸價格在1.14萬元、型鋼每公噸2.7萬元。國際原料行情，美國大船廢鋼本週無報價，日本2H廢鋼本週上漲至480美元/噸，美國貨櫃廢鋼本週報價上漲至450美元/噸，澳


文章標題:【台股盤中】電子航運領軍 台股盤中站回16700點（中央社 2021-08-23）
內文:...570元，大漲3.26%。聯發科(2454)來到910元，上漲4.42%。大立光(3008)來到2810元，上漲2.36%。鴻海(2317)107元，上漲2.39%。貨櫃三雄長榮(2603)盤中


文章標題:《台北股市》盤中焦點股：台積電、新鋼、慧洋-KY、萬海、世芯-KY（時報資訊 2021-08-23）
內文:...上周五外資逆勢轉買超2953張，加上ADR漲1.67%，今早股價彈升近4%、躍過5日線。 2.台積電(2330)：外資賣超不停歇、上周五ADR跌2.17%，惟投信買超力挺，加上高盛證券送暖重申買


文章標題:【Y早報】微軟、思科領軍科技股 美四大指數收紅（Yahoo奇摩股市 2021-08-23）
內文:（開盤日09:00出刊）晶片備貨旺季，封測產能吃緊，3測試廠旺到明年；當沖降稅延長，牽動權證避險降稅 ，已釋出2版本；台股進入超跌區，救市資金偏愛16檔具3條件股。


文章標題:《鋼鐵股》新光鋼 今年EPS挑戰9元（時報資訊 2021-08-23）
內文:...工程需求榮景持續加持下，目前下半年仍然穩定接單，且加工製造訂單可望延續到2022上半